In [1]:
%reset -fs

In [2]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
import pandas as pd
import datetime

In [3]:
client = MongoClient()

In [4]:
db = client.nba_db
cursor = db.betting_data.find()
mongo_df = pd.DataFrame(list(cursor))
mongo_df

,_id,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
0,6246960a4fba57b3e39e9976,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,15.5,11.0,o,0.47,0.0
1,6246960a4fba57b3e39e9977,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5.0,o,0.45,0.0
2,6246960a4fba57b3e39e9978,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,15.5,14.0,u,0.45,1.0
3,6246960a4fba57b3e39e9979,2021-10-27,Bogdan,ATL,G,NOR,@,12.5,-125,13.70,1.20,12.5,12.0,o,0.45,0.0
4,6246960a4fba57b3e39e997a,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14.0,o,0.42,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14507,6246960a4fba57b3e39ed221,2022-03-08,Ziaire Williams,MEM,G,NOR,vs.,6.5,-110,7.30,0.80,6.5,16.0,o,0.46,1.0
14508,6246960a4fba57b3e39ed222,2022-03-11,Ziaire Williams,MEM,G,NYK,vs.,7.5,-115,7.30,-0.20,7.5,13.0,u,0.50,0.0
14509,6246960a4fba57b3e39ed223,2022-03-15,Ziaire Williams,MEM,G,IND,@,8.5,-130,7.60,-0.90,8.5,11.0,u,0.46,0.0
14510,6246960a4fba57b3e39ed224,2022-03-23,Ziaire Williams,MEM,G,BKN,vs.,7.5,-105,6.20,-1.30,7.5,0.0,u,0.46,1.0


In [5]:
mongo_df[mongo_df.Date == '2022-03-31']

,_id,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy,successful_pick
50,6246960a4fba57b3e39e99a8,2022-03-31,Bogdan,ATL,G,CLE,vs.,16.5,-115,14.80,-1.70,16.5,NaN,u,0.46,NaN
351,6246960a4fba57b3e39e9ad5,2022-03-31,Alex Caruso,CHI,G,LAC,vs.,7.5,-125,8.73,1.23,7.5,NaN,o,0.46,NaN
485,6246960a4fba57b3e39e9b5b,2022-03-31,Andre Drummond,BKN,C,MIL,vs.,10.5,-115,11.20,0.70,10.5,NaN,o,0.46,NaN
739,6246960a4fba57b3e39e9c59,2022-03-31,Austin Reaves,LAL,G,UTH,@,7.5,-110,8.80,1.30,7.5,NaN,o,0.46,NaN
832,6246960a4fba57b3e39e9cb6,2022-03-31,Ayo Dosunmu,CHI,G,LAC,vs.,9.5,105,9.80,0.30,9.5,NaN,o,0.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13841,6246960a4fba57b3e39ecf87,2022-03-31,Trae Young,ATL,G,CLE,vs.,29.5,-108,27.00,-2.50,29.5,NaN,u,0.46,NaN
13971,6246960a4fba57b3e39ed009,2022-03-31,Tristan Thompson,CHI,C,LAC,vs.,4.5,-120,5.19,0.69,4.5,NaN,o,0.46,NaN
14180,6246960a4fba57b3e39ed0da,2022-03-31,Tyrese Maxey,PHI,G,DET,@,16.5,-113,18.80,2.30,16.5,NaN,o,0.46,NaN
14314,6246960a4fba57b3e39ed160,2022-03-31,Wesley Matthews,MIL,G,BKN,@,5.5,-115,5.10,-0.40,5.5,NaN,u,0.46,NaN


In [15]:
today = datetime.datetime.today()
max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')

def get_date_list(date):
    date_list = []
    today = datetime.datetime.today()
    max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')
    delta = today - max_date
    date_list = [max_date - datetime.timedelta(days=x) for x in range(0, delta.days)]
    date_list = [d.strftime('%Y-%m-%d') for d in date_list]
    date_list.append((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d'))
    date_list.append(datetime.datetime.today().strftime('%Y-%m-%d'))
    date_list = set(date_list)
    date_list = list(date_list)
    return date_list
    
    
    

In [17]:
date_list = get_date_list(today)
date_list


['2022-04-01', '2022-03-31']

In [18]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [19]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [20]:
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [21]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df

In [22]:
final_df = pd.DataFrame()

In [23]:
for i in date_list:
    get_site_rb(i)

In [24]:
driver.close()

In [25]:
final_df

,0,1,2,3,4,5,6,7,8,9
0,1 T. Theo Maledon,OKC - G,vs. DET,Points,18.50 (-122)1.72Diff: -16.78 Line: 18.5...,18.5,-122 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
1,2 I. Isaiah Roby,OKC - F,vs. DET,Points,14.50 (-112)4.96Diff: -9.54 Line: 14.50...,14.5,-112 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
2,3 A. Aleksej Pokusevski,OKC - C,vs. DET,Points,15.50 (-113)6.08Diff: -9.42 Line: 15.50...,15.5,-113 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
3,4 D. Damian Jones,SAC - C,@ HOU,Points,14.50 (-120)5.57Diff: -8.93 Line: 14.50...,14.5,-120 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
4,5 D. Davion Mitchell,SAC - G,@ HOU,Points,22.50 (-113)14.86Diff: -7.64 Line: 22.5...,22.5,-113 TBD,TBD,04-01-2022,Last 7 days Top 15 Accuracy: 36-51 (41%) Sea...
...,...,...,...,...,...,...,...,...,...,...
13,64 A. Ayo Dosunmu,CHI - G,vs. LAC,Points,9.50 (+105)9.80Diff: 0.30 Line: 9.50 Co...,9.5,+105 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
14,65 M. Marvin Bagley III,DET - F,vs. PHI,Points,15.50 (-107)15.80Diff: 0.30 Line: 15.50...,15.5,-107 TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
15,66 R. Reggie Jackson,LAC - G,@ CHI,Points,17.50 (EVEN)17.30Diff: -0.20 Line: 17.5...,17.5,EVEN TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...
16,67 B. Bruce Brown Jr.,BKN - G,vs. MIL,Points,9.50 (EVEN)9.30Diff: -0.20 Line: 9.50 C...,9.5,EVEN TBD,TBD,03-31-2022,Last 7 days Top 15 Accuracy: 46-55 (46%) Sea...


In [26]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [27]:
final_df.reset_index(drop=True, inplace=True)

In [28]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [29]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [30]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [31]:
bet_info = final_df.Line.str.split(" ", expand = True)
bet_info

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,,,,,,,,18.50,(-122)1.72Diff:,-16.78,Line:,18.50,Cost:,-122,Proj.:,1.72,u
1,,,,,,,,14.50,(-112)4.96Diff:,-9.54,Line:,14.50,Cost:,-112,Proj.:,4.96,u
2,,,,,,,,15.50,(-113)6.08Diff:,-9.42,Line:,15.50,Cost:,-113,Proj.:,6.08,u
3,,,,,,,,14.50,(-120)5.57Diff:,-8.93,Line:,14.50,Cost:,-120,Proj.:,5.57,u
4,,,,,,,,22.50,(-113)14.86Diff:,-7.64,Line:,22.50,Cost:,-113,Proj.:,14.86,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,,,,,,,,9.50,(+105)9.80Diff:,0.30,Line:,9.50,Cost:,+105,Proj.:,9.80o,None
99,,,,,,,,15.50,(-107)15.80Diff:,0.30,Line:,15.50,Cost:,-107,Proj.:,15.80o,None
100,,,,,,,,17.50,(EVEN)17.30Diff:,-0.20,Line:,17.50,Cost:,EVEN,Proj.:,17.30,u
101,,,,,,,,9.50,(EVEN)9.30Diff:,-0.20,Line:,9.50,Cost:,EVEN,Proj.:,9.30,u


In [32]:
bet_info = final_df.Line.str.split(" ", expand = True)
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [33]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [34]:
final_df = final_df.copy()

In [35]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [36]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [37]:
final_df['seven_day_accuracy'] = accuracies[8]

In [38]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [39]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [40]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [41]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [42]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [43]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [44]:
final_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2022-04-01,Theo Maledon,OKC,G,DET,vs.,18.50,-122,1.72,-16.78,18.5,TBD,u,0.41
1,2022-04-01,Isaiah Roby,OKC,F,DET,vs.,14.50,-112,4.96,-9.54,14.5,TBD,u,0.41
2,2022-04-01,Aleksej Pokusevski,OKC,C,DET,vs.,15.50,-113,6.08,-9.42,15.5,TBD,u,0.41
3,2022-04-01,Damian Jones,SAC,C,HOU,@,14.50,-120,5.57,-8.93,14.5,TBD,u,0.41
4,2022-04-01,Davion Mitchell,SAC,G,HOU,@,22.50,-113,14.86,-7.64,22.5,TBD,u,0.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2022-03-31,Ayo Dosunmu,CHI,G,LAC,vs.,9.50,+105,9.80,0.30,9.5,TBD,o,0.46
99,2022-03-31,Marvin Bagley,DET,F,PHI,vs.,15.50,-107,15.80,0.30,15.5,TBD,o,0.46
100,2022-03-31,Reggie Jackson,LAC,G,CHI,@,17.50,EVEN,17.30,-0.20,17.5,TBD,u,0.46
101,2022-03-31,Bruce Brown,BKN,G,MIL,vs.,9.50,EVEN,9.30,-0.20,9.5,TBD,u,0.46


In [45]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [46]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [47]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [48]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [49]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x));sorted_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,over_under,seven_day_accuracy
0,2022-03-31,Bogdan,ATL,G,CLE,vs.,16.5,-115,14.80,-1.70,16.5,TBD,u,0.46
1,2022-04-01,Aaron Gordon,DEN,F,MIN,vs.,14.5,-113,16.69,2.19,14.5,TBD,o,0.41
2,2022-04-01,Al Horford,BOS,C,IND,vs.,9.5,100,9.33,-0.17,9.5,TBD,u,0.41
3,2022-04-01,Aleksej Pokusevski,OKC,C,DET,vs.,15.5,-113,6.08,-9.42,15.5,TBD,u,0.41
4,2022-03-31,Alex Caruso,CHI,G,LAC,vs.,7.5,-125,8.73,1.23,7.5,TBD,o,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,2022-03-31,Tristan Thompson,CHI,C,LAC,vs.,4.5,-120,5.19,0.69,4.5,TBD,o,0.46
99,2022-03-31,Tyrese Maxey,PHI,G,DET,@,16.5,-113,18.80,2.30,16.5,TBD,o,0.46
100,2022-03-31,Wesley Matthews,MIL,G,BKN,@,5.5,-115,5.10,-0.40,5.5,TBD,u,0.46
101,2022-04-01,Will Barton,DEN,G,MIN,vs.,15.5,-113,17.17,1.67,15.5,TBD,o,0.41


In [50]:
results = sorted_df.Result.str.split(' ', expand=True);

In [51]:
results

,0
0,TBD
1,TBD
2,TBD
3,TBD
4,TBD
...,...
98,TBD
99,TBD
100,TBD
101,TBD


In [52]:
try:
    sorted_df.Result = results[3]
except:
    sorted_df.Result = results[0]
sorted_df.Result

0      TBD
1      TBD
2      TBD
3      TBD
4      TBD
      ... 
98     TBD
99     TBD
100    TBD
101    TBD
102    TBD
Name: Result, Length: 103, dtype: object

In [55]:
sorted_df = sorted_df.replace('TBD', None)

In [56]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [57]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)